In [1]:
import torch
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import os
import pandas
from dataset import getPerformancesList
import h5py
%matplotlib inline

In [2]:
SNAPSHOTS_DIR = "/home/pasinducw/Documents/research/university-work-scs-4224/comparing-rnn-params/model/model-snapshots/exp15"
SONGS_DIR = "/home/pasinducw/Downloads/Research-Datasets/covers80/old/covers80_cqt/"

In [3]:
excluded_transforms = [
        "_PITCH_SHIFT_0", "_PITCH_SHIFT_1", "_PITCH_SHIFT_2", "_PITCH_SHIFT_3", "_PITCH_SHIFT_4",
        "_TIME_STRETCH_0", "_TIME_STRETCH_1", "_TIME_STRETCH_2", "_TIME_STRETCH_3", "_TIME_STRETCH_4",
    ]

# Object array of shape {song: string, name: string, path: string} containing the details of performances
PERFORMANCES = getPerformancesList(root_dir = SONGS_DIR, excluded_transforms = excluded_transforms)

In [4]:
# Array containing the songs
SONGS = []
for performance in PERFORMANCES:
    SONGS.append(performance['song'])
SONGS = np.unique(SONGS)

In [14]:
# Add model parameters to the PERFORMANCES objects
# Resulting object -> { song: string, name: string, path: string, snapshot: OrderedDict, parameters: vector }

for (index, performance) in enumerate(PERFORMANCES):
    snapshot_location = os.path.join(
        SNAPSHOTS_DIR,
        '{}-{}'.format(index, performance['name']),
        'snapshot-400.pytorch'
    )
    snapshot = torch.load(snapshot_location)['model_state_dict']
    performance['snapshot'] = snapshot # This is an ordered dictionary with model parameters
    
    parameters = None
    for key in snapshot.keys():
        values = snapshot[key].flatten()
        if parameters is None:
            parameters = values
        else:
            parameters = torch.cat((parameters, values))
    performance['parameters'] = parameters # This is a vector that has all the parameters of the model flattened

In [34]:
DATA_PATH = "/home/pasinducw/Downloads/Research-Datasets/covers80/old/embeddings"

# performance = PERFORMANCES[0]
count =0
for performance in PERFORMANCES:
  count += 1
  track_id = performance['name']
  work_id = performance['song']
  path = os.path.join(DATA_PATH, work_id)
  if not os.path.exists(path):
      os.makedirs(path)
  path = os.path.join(path, "{}.{}".format(track_id, 'h5'))
#   print(performance['parameters'].numpy().dtype)

  with h5py.File(path, "w") as f:
      embedding = f.create_dataset("embedding", data = performance['parameters'].numpy())
print(count)


1312


In [ ]:
# Create the metadata
METADATA_PATH = "/home/pasinducw/Downloads/Research-Datasets/covers80/old/embeddings"
path = os.path.join(METADATA_PATH, "metadata.csv")
with open(path, "w") as f:
  f.write("work_id,track_id\n")

  for performance in PERFORMANCES:
    f.write("{},{}\n".format(performance['song'], performance['name']))

In [37]:
# Verify the data
DATA_PATH = "/home/pasinducw/Downloads/Research-Datasets/covers80/old/embeddings"
performance = PERFORMANCES[0]

track_id = performance['name']
work_id = performance['song']
path = os.path.join(DATA_PATH, work_id)
path = os.path.join(path, "{}.{}".format(track_id, 'h5'))
print(path)

with h5py.File(path, "r") as f:
  print(list(f.keys()))
  data = f['embedding'][:]

  
print(data)

/home/pasinducw/Downloads/Research-Datasets/covers80/old/embeddings/A_Whiter_Shade_Of_Pale/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale_WAV.h5
['embedding']
[-0.04271855  0.55806744 -0.0234014  ... -0.12250717 -0.05396162
  0.21084808]
